In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
N, n = 21, 6
A, b = np.zeros((n, n), dtype=float), np.full(n, fill_value=N, dtype=float)

for i in range(n):
    for j in range(n):
        c = 0.1 * N * (i + 1) * (j + 1)
        A[i, j] = 1000 / (3*c**2 + c**3)

x = np.linalg.solve(A, b)
cond_value = np.linalg.cond(A, p=np.inf)
print(f"Решение системы {x=}, Число обусловленности = {cond_value}")

In [5]:
delta = 0.05
d = np.empty(n)
for i in range(n):
    b_i = b.copy()
    b_i[i] += delta
    x_i = np.linalg.solve(A, b_i)
    d[i] = np.linalg.norm(x - x_i, ord=np.inf) / np.linalg.norm(x, ord=np.inf)

print(f"Вектор относительных погрешностей {d=}")


In [6]:
plt.bar(np.arange(0, n), d)
plt.title("Гистограмма вектора d")
print(f"Компонента b {b[np.argmax(d)]}, оказывающая наибольшее влияние, имеет индекс {np.argmax(d)}")

In [7]:
for i in range(n):
    b_i = b.copy()
    b_i[i] += delta

    estimation = cond_value * (np.linalg.norm(b - b_i, ord=np.inf) / np.linalg.norm(b, ord=np.inf))
    print(f"x{i}: Погрешность detla(xi): {d[i]}, Оцененная погрешность delta(xi): {estimation}")

In [8]:
import pandas as pd

n, m = 25, 10

def cholesky(A: np.ndarray, n: int) -> np.ndarray:
    L = np.zeros((n,n))

    for j in range(n):
        for i in range(n):
            if j < i:
                sum = np.dot(L[i,:], L[j,:])
                multiplier = 1/L[j,j]
                sum = 0 if np.isnan(sum) or np.isinf(sum) else sum
                multiplier = 1 if np.isnan(multiplier) or np.isinf(multiplier) else multiplier

                L[i,j] = multiplier * (A[i,j] - sum)
            if i == j:
                sum = np.sum(L[i,:i]**2)
                sum = 0 if np.isnan(sum) or np.isinf(sum) else sum

                L[i,j] = np.sqrt(A[i,i] - sum)
    return L

In [9]:
A = np.empty((n,n))
b = np.arange(n)**2 - n

for i in range(n):
    for j in range(n):
        if i == j:
            A[i,j] = n + m**2 + j/m + i/n
        else:
            A[i,j] = (i+j)/(m+n)

L = cholesky(A,n)
A_tmp = L.dot(L.transpose())   # Проверка решения
print(np.allclose(A, A_tmp))
pd.DataFrame(L).to_csv('lmatrix.csv')
pd.DataFrame(L.transpose()).to_csv('lmatrix.csv')

In [10]:
y = np.linalg.solve(L, b)
x = np.linalg.solve(L.transpose(), y)
print(x)

In [11]:
def value_to_decimal(value, m):
    return int(value * 10 ** m + 0.5) / 10 ** m

In [12]:
def gauss_partial_sle(A, b):
    n = len(b)
    M = np.hstack((A, b.reshape(-1, 1)))
    # Прямой ход
    for k in range(n):
        # Выбор главной строки по столбцу
        max_index = k + np.argmax(np.abs(M[k:,k]))
        M[k], M[max_index] = M[max_index].copy(), M[k].copy()
        for i in range(k+1, n):
            modifier = M[i, k] / M[k, k]
            M[i, :] -= M[k, :] * modifier

    # Обратный ход
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        multiplier = 1 / M[i,i]
        sum = np.dot(M[i,i+1:n], x[i+1:])
        x[i] = multiplier * (M[i, -1] - sum)
    return x

In [13]:
def gauss_partial_sle_rounded(A, b):
    n = len(b)
    M = np.hstack((A, b.reshape(-1, 1)))
    for k in range(n):
        max_index = k + np.argmax(np.abs(M[k:,k]))
        M[k], M[max_index] = M[max_index].copy(), M[k].copy()

        for i in range(k+1, n):
            modifier = value_to_decimal(M[i, k] / M[k, k], m)
            M[i, :] -= M[k, :] * modifier
            M[i, :] = np.array([value_to_decimal(v, m) for v in M[i, :]])

    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        multiplier = value_to_decimal(1 / M[i,i], m)
        sum = np.dot(M[i,i+1:n], x[i+1:])
        x[i] = value_to_decimal(multiplier * (M[i, -1] - sum), m)
    return x

In [14]:
def generate_sle(t):
    qm = 0.993 + (-1)**M * M * 10 ** -4

    A = np.empty((n,n))
    b = qm**(n+1-np.arange(n))
    for i in range(n):
        for j in range(n):
            q_j = qm ** j
            if i == j:
                A[i,j] = q_j + t
            else:
                A[i,j] = q_j
    return A, b


def generate_sle_rounding(t):
    qm = np.round(0.993 + np.round((-1)**M * M * 10 ** -4, m),m)

    A = np.empty((n,n))
    b = np.round(qm**(n+1-np.arange(n)),m)
    for i in range(n):
        for j in range(n):
            q_j = np.round(qm ** j, m)
            if i == j:
                A[i,j] = np.round(q_j + np.round(t,m))
            else:
                A[i,j] = q_j
    return A, b


In [19]:
M, N, n, m = 5, 5, 45, 4
T = [0.0001, 1, 10000]
err = np.empty(len(T))
abserr = np.empty(len(T))
disc = np.empty(len(T))

for t, i in zip(T, np.arange(len(T))):
    A1, b1 = generate_sle(t)
    A2, b2 = generate_sle_rounding(t)
    x1 = gauss_partial_sle(A1, b1)
    x2 = gauss_partial_sle_rounded(A2,b2)
    discrepancy1 = b2 - A2.dot(x2)
    # print(f"t: {t}; Решение системы {x1=}")
    # print(f"t: {t}; Решение системы {x2=}")
    err[i] = np.linalg.norm(x1 - x2) / np.linalg.norm(x1)
    abserr[i] = np.linalg.norm(x1 - x2)
    disc[i] = np.linalg.norm(discrepancy1)
    print(f"t: {t};\n Относительная погрешность решения: {np.linalg.norm(x1 - x2) / np.linalg.norm(x1)},\n Норма неувязки второго решения: {np.linalg.norm(discrepancy1)}")
    print()

In [23]:
plt.plot(np.arange(len(T)), err)
plt.xticks(np.arange(len(T)), T)
plt.title("Относительная погрешность")

In [24]:
plt.plot(np.arange(len(T)), abserr)
plt.xticks(np.arange(len(T)), T)
plt.title("Абсолютная погрешность")


In [25]:
plt.plot(np.arange(len(T)), disc)
plt.xticks(np.arange(len(T)), T)
plt.title("Невязка")
